# _AutoAuto_ Self-driving Car

Let's use _Machine Learning_ (specifically, an _Artificial Neural Network_) to drive the _AutoAuto_ car around a track.

![](images/car.jpg)

### What is Machine Learning?

_Machine Learning_ (ML) is a subfield of _Artificial Intelligence_. ML algorithms use training data to learn patters. **We** write the learning algorithm, then we feed training data into the algorithm so that it can "learn".

ML is useful for:
1. Having our computers detect patters which we humans **can't** detect, or
2. Having our computers do tasks which are **boring** for us to do.

Read more on [Wikipedia](https://en.wikipedia.org/wiki/Machine_learning).

![](images/ml.jpg)

### Training Data

We collected over 8,000 images by manually driving the car around on the track. Every time an image was taken, the angle of the wheels was also recorded! That means we have over 8,000 image/angle pairs which were used to train the machine learning algorithm.

Here are a few examples from the training set. The number above each image denotes the angle of the wheels at the moment the image was taken (positive is _left_, negative is _right_).

![](images/samples.png)

### Preprocessing

Before using the training images, we _preprocess_ them to remove some of the noise which is irrelevent for driving.

See below how each of the examples images is preprocessed. Notice how we:
1. crop the image,
2. remove the color and,
3. only keep strong edges!

![](images/samples_preprocessed.png)

### What is an Artificial Neural Network?

An _Artificial Neural Network_ is one machine learning algorithm which usually does a good job of learning from image data like we have here!

Neural Networks contain many little neurons which are connected together into layers. Using an algorithm nicknamed "backprop", the neural network learns from the training data.

Read more on [Wikipedia](https://en.wikipedia.org/wiki/Machine_learning#Artificial_neural_networks).

![](images/ann_alt.png)

### Load the necessary Python libraries

The next cell loads the TensorFlow library. TensorFlow is a library which does very fast numeric operations. (Thanks Google!)

In [ ]:
TF_NUM_THREADS = 1     # <-- the number of threads we want tensorflow to use

from keras import backend as K
import tensorflow as tf

config = tf.ConfigProto(intra_op_parallelism_threads=TF_NUM_THREADS, \
                        inter_op_parallelism_threads=TF_NUM_THREADS, \
                        allow_soft_placement=True, \
                        device_count = {'CPU': TF_NUM_THREADS})
session = tf.Session(config=config)
K.set_session(session)

The next cell loads the `OpenCV` library. OpenCV is used for doing image processing and computer vision. (Thanks Intel!)

In [ ]:
import numpy as np
import cv2

cv2.setNumThreads(0)   # <-- disable multithreading in opencv

The next cell loads the car library! (Thanks AutoAuto!)

In [ ]:
import time

import car
from car.camera import CameraRGB
from car import throttle
from car import steering

### Load the Neural Network!

The next cell loads the pre-trained Neural Network!

In [ ]:
from keras.models import load_model
model = load_model("model_01.hdf5")

This next cell creates functions to do the preprocessing on the image, and it starts the car's camera.

In [ ]:
def crop(img):
    height = 70
    return img[-height:, :]

def edges(img):
    canny_thresh_1, canny_thresh_2 = 100, 200
    return cv2.Canny(img, canny_thresh_1, canny_thresh_2)

def preprocess(img):
    img_edge = edges(crop(img))
    img_edge = np.expand_dims(img_edge, 2)
    img_edge = np.array(img_edge, dtype=np.float32) / 255.0
    return img_edge

camera = CameraRGB(height=128, width=160, fps=8)

### Finally, Drive the Car!

In [ ]:
from IPython.display import clear_output

throttle.set_throttle(0)
steering.set_steering(0)

try:
    while True:
        frame = camera.capture()
        car.stream(frame)
        frame = preprocess(frame)
        steering_angle = model.predict(np.array([frame]))[0][0] * 90.
        steering.set_steering(steering_angle)
        clear_output(True)
        print("Angle:", steering_angle)
        throttle.set_throttle(50)
        time.sleep(0.25)
        throttle.set_throttle(0)
        time.sleep(0.05)

except KeyboardInterrupt:
    pass

throttle.set_throttle(0)
steering.set_steering(0)

# Appendix -- Do not present this.

The code below is used to make a nice plot showing some example images from the training set + the same images after preprocessing.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

with open("data4/X32.npy", 'rb') as f:
    X = np.load(f)
with open("data4/y32.npy", 'rb') as f:
    y = np.load(f)
    
print(X.shape)

p = [613, 785, 494, 755, 791]  # np.random.permutation(len(X))[:5]
print(p)
fig, axes = plt.subplots(1, 5, figsize=(15,5))

white = (np.ones((40, 160, 3)) * 255).astype(np.uint8)

for img, label, ax in zip(X[p] ,y[p], axes.flatten()):
    pcd = np.array(cv2.cvtColor(preprocess(img) * 255., cv2.COLOR_GRAY2RGB), dtype=np.uint8)
    ax.imshow(np.concatenate((img, white, pcd), axis=0))
    ax.axis('off')
    ax.set_title(str(int(round(label))))

fig.suptitle("5 Example Training Images")